# MOC-TEML:
## CASO DE ESTUDIO: PREDICCIÓN DE TENDENCIA EN LOS ÍNDICES BURSÁTILES DE LA BOLSA DE VALORES DE COLOMBIA
### Índice Bursátil: COLCAP


# Importación de librerías

Se importan las librerías necesarias para el tratamiento de datos y la implementación de los modelos.

In [ ]:

import pandas as pd
import numpy as np 
import datetime
import matplotlib.pyplot as plt

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 4)
plt.style.use('fast')

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


# Selección de información

Se importa la información desde el archivo de origen y se construye un dataframe con la incormación del índice bursátil analizado.

In [ ]:
#from pylab import *
#from google.colab import files

In [ ]:
#!rm 'Indices Consolidado.xlsx'
#uploaded = files.upload()

In [ ]:
#import io
#df = pd.read_excel(io.BytesIO(uploaded['Indices Consolidado.xlsx']),sheet_name='Data', index_col='Fecha 3', parse_dates=True)

In [ ]:
df=pd.read_excel('Indices Consolidado.xlsx',sheet_name='Data', index_col='Fecha 3', parse_dates=True) 
df = df[df['Indice']=='COLCAP']
df=df.drop(columns=['Source.Name', 'Fecha', 'Fecha 2', 'Indice', 'Valor Ayer', 'Variacion %', 'Variacion Absoluta', 'Variacion 12 meses', 'Variacion Anual'])
print(df.shape)

#### Se visualiza un resumen de la información importada y se presenta la gráfica de la serie de tiempo a analizar.

In [ ]:
df.describe().transpose()

In [ ]:
df.head()

In [ ]:
plt.plot(df.values)

# Preparar el Data Set

Se prepara el data set, teniendo en cuenta que el codelo corresponde a una serie de tiempo se construye un vector de entrada que constituye en los 6 días anteriores para pronosticar el día 7, teniendo en cuenta una semana como referencia.
El dataframe se conforma con una columna por cada día de la semana anterior al día pronosticado.
Finalmente se normalizan los datos con el min max scaler.

In [ ]:
PASOS=6


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    if dropnan: 
        agg.dropna(inplace=True)
    return agg
 


values = df.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(-1, 1))
values=values.reshape(-1, 1) 
scaled = scaler.fit_transform(values)

reframed = series_to_supervised(scaled, PASOS, 1)
reframed.head()


# Dividir los datos en conjuntos de entrenamiento y testeo

Se realiza la división de los datos en el conjunto de entrenamiento y de testeo.

In [ ]:
values = reframed.values

data_long = reframed.shape
data_long = data_long[0]

model_dataset_long = 0.75

model_dataset_long = round(data_long*model_dataset_long )

model_values = values[:model_dataset_long, :]
test_values = values[model_dataset_long:, :]

In [ ]:
print(reframed.shape, model_values.shape, test_values.shape)

In [ ]:
split_train = 10
train_dataset= 1-(1/split_train)

model_data_long = model_values.shape
model_data_long = model_data_long[0]

n_train_days = round(model_data_long*train_dataset )
train = model_values[:n_train_days, :]
test = model_values[n_train_days:, :]

x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]

x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

# Seleccionar técnica de modelado

Se modela el problema como una serie de tiempo donde se considera que el valor del índice varía en el tiempo. Es decir que corresponde a un problema de aprendizaje supervisado de regresión. Los vectores de entrada corresponden al valor del índice en los 6 días inmediatamente anteriores.

# Seleccionar Algoritmo 1 - ANN

El primer modelo corresponde a una red neuronal artificial del tipo perceptrón multicapa.

# Definir Hiperparámetros y crear modelo

Para este perceptrón multicapa se define una red neuronal de la siguiente forma:
    - Una capa de entrada conformada por 6 neuronas dados los 6 días anteriores de referencia.
    - Una capa oculta con 6 neurona y una función de activación tangenete hiperbólica.

In [ ]:
def crear_modeloFF():
    model = Sequential() 
    model.add(Dense(PASOS, input_shape=(1,PASOS),activation='tanh'))
    model.add(Flatten())
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model

# Entrenamiento del algoritmo

Se entrena la red neuronal con la parametrización creada y en 100 épocas.

In [ ]:
EPOCHS=100

model = crear_modeloFF()

history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)

## Se realiza la predicción con los datos de testeo

Se presentan unos resultados iniciales del entrenamiento normalizado 

In [ ]:
results=model.predict(x_val)
plt.scatter(range(len(y_val)),y_val,c='g')
plt.scatter(range(len(results)),results,c='r')
plt.title('validate')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('loss')
plt.plot(history.history['val_loss'])
plt.title('validate loss')
plt.show()

In [ ]:
#plt.title('Accuracy')
#plt.plot(history.history['mean_squared_error'])
#plt.show()

In [ ]:
print(y_val.shape,results.shape)

# Crossvalidation

Se implementa la validación cruzada con la técnica KFold definiendo 10 splits

In [ ]:
from sklearn.model_selection import KFold

values = reframed.values
x, y = values[:, :-1], values[:, -1]

kf=KFold(n_splits=split_train, shuffle=True, random_state=2)
for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index],x[val_index]
    y_train, y_val = y[train_index],y[val_index]
    
    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
    
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
    
    EPOCHS=100
    history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)
    
    results=model.predict(x_val)
    

In [ ]:
results=model.predict(x_val)
plt.scatter(range(len(y_val)),y_val,c='g')
plt.scatter(range(len(results)),results,c='r')
plt.title('validate')
plt.show()

### Predicción con los datos de testeo

In [ ]:

x_test, y_test = test_values[:, :-1], test_values[:, -1]

x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

results_test_ann=model.predict(x_test)

#### Se desnormalizan los datos de la predicción y se construye un Data Frame con el valor del resultado real, el pronóstico obtenido y la diferencia.

In [ ]:
compara = pd.DataFrame(np.array([y_test, [x[0] for x in results_test_ann]])).transpose()
compara.columns = ['real', 'prediccion']

inverted = scaler.inverse_transform(compara.values)

compara_ann = pd.DataFrame(inverted)
compara_ann.columns = ['real', 'prediccion_ann']
compara_ann['diferencia_ann'] = compara_ann['real'] - compara_ann['prediccion_ann']
compara_ann.head()

In [ ]:
compara_ann.describe()

#### Sepresenta graficamente los resultados de la comparación

In [ ]:
compara_ann['real'].plot()
compara_ann['prediccion_ann'].plot()

# Seleccionar Algoritmo 2 - SVR

El segundo algoritmo corresponde a support vector machine de regresión o más conocido como SVR.

In [ ]:
data_long = reframed.shape
data_long = data_long[0]

train_dataset = 0.75

values = reframed.values
n_train_days = round(data_long*train_dataset )
train = values[:n_train_days, :]
test = values[n_train_days:, :]

x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]

x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)


train_long, val_long = x_train.shape, x_val.shape
train_long, val_long = train_long[0], val_long[0]
y_train=y_train.reshape((train_long,1))
x_train=x_train.reshape((train_long,PASOS))
x_val=x_val.reshape((val_long,PASOS))
y_val=y_val.reshape((val_long,1))


# Definir de Función Kernel

Para este algoritmo se selecciona como Kernel la función en base radial (RBF).

In [ ]:
from sklearn.svm import SVR 
from sklearn.metrics import mean_squared_error 
import math

#model = SVR(C=0.5, kernel='poly', degree=1)
model = SVR()
print(model)

# Entrenamiento del algoritmo

Se entrena la red neuronal con la parametrización creada y en 100 épocas.

In [ ]:

model.fit(x_train,y_train)


pred_y = model.predict(x_val)

score=model.score(x_val,y_val)
print(score)

mse =mean_squared_error(y_val, pred_y)
print("Mean Squared Error:",mse)

rmse = math.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# Se realiza la predicción con los datos de testeo

Se presentan unos resultados iniciales del entrenamiento normalizado 

In [ ]:
pred_y=pred_y.reshape((val_long,1))

In [ ]:

plt.scatter(range(len(y_val)),y_val,c='g')
plt.scatter(range(len(pred_y)),pred_y,c='r')
plt.title('validate')
plt.show()

In [ ]:
y_val=y_val.reshape((val_long,))
results=pred_y
print(y_val.shape, results.shape)

# Crossvalidation

Se implementa la validación cruzada con la técnica KFold definiendo 10 splits

In [ ]:
from sklearn.model_selection import KFold

values = reframed.values
x, y = values[:, :-1], values[:, -1]

kf=KFold(n_splits=10, shuffle=True, random_state=2)
for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index],x[val_index]
    y_train, y_val = y[train_index],y[val_index]
    
    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
    
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
    
    train_long, val_long = x_train.shape, x_val.shape
    train_long, val_long = train_long[0], val_long[0]
    y_train=y_train.reshape((train_long,1))
    x_train=x_train.reshape((train_long,PASOS))
    x_val=x_val.reshape((val_long,PASOS))
    y_val=y_val.reshape((val_long,1))
    
    model.fit(x_train,y_train)
    
    pred_y = model.predict(x_val)
    
    score=model.score(x_val,y_val)
    print(score)
    
    mse =mean_squared_error(y_val, pred_y)
    print("Mean Squared Error:",mse)
    
    rmse = math.sqrt(mse)
    print("Root Mean Squared Error:", rmse)

In [ ]:
pred_y=pred_y.reshape((val_long,1))
plt.scatter(range(len(y_val)),y_val,c='g')
plt.scatter(range(len(pred_y)),pred_y,c='r')
plt.title('validate')
plt.show()

In [ ]:
y_val=y_val.reshape((val_long,))
results=pred_y
print(y_val.shape, results.shape)

### Predicción con los datos de test

In [ ]:

x_test, y_test = test_values[:, :-1], test_values[:, -1]
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

test_long= x_test.shape
test_long = test_long[0]
y_test = y_test.reshape((test_long,1))
x_test = x_test.reshape((test_long,PASOS))

pred_y = model.predict(x_test)

pred_y=pred_y.reshape((test_long,1))


y_test=y_test.reshape((test_long,))
results_test_svr=pred_y
print(y_test.shape, results_test_svr.shape)



#### Se desnormalizan los datos de la predicción y se construye un Data Frame con el valor del resultado real, el pronóstico obtenido y la diferencia.

In [ ]:
compara = pd.DataFrame(np.array([y_test, [x[0] for x in results_test_svr]])).transpose()
compara.columns = ['real', 'prediccion']

inverted = scaler.inverse_transform(compara.values)

compara_svr = pd.DataFrame(inverted)
compara_svr.columns = ['real', 'prediccion_svr']
compara_svr['diferencia_svr'] = compara_svr['real'] - compara_svr['prediccion_svr']
compara_svr.head()

In [ ]:
compara_svr.describe()

In [ ]:
compara_svr['real'].plot()
compara_svr['prediccion_svr'].plot()

# Comparación de los datos

Se calcula la matriz de desviación para determinar el modelo más preciso hallando la menor desviación media.

In [ ]:
resultado_modelos = compara_ann.merge(compara_svr, left_index=True, right_index=True)

### Tabla de resultados de los modelos aplicados

In [ ]:
resultado_modelos.head()

### Comparación Gráfica de los modelos aplicados

In [ ]:
resultado_modelos['real_y'].plot(c='black')
resultado_modelos['prediccion_ann'].plot(c='r')
resultado_modelos['prediccion_svr'].plot(c='b')

# Propiedades de la imagen
plt.rcParams["figure.figsize"] = 8, 4
legend(('Real', 'ANN', 'SVR'),
  prop = {'size': 10}, loc='upper right',
  bbox_to_anchor=(0.2,0.4))

xlabel('Días')
ylabel('Valor / $')
title('Comparación modelos aplicados para índice COLCAP')

plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
mse_ann=mean_squared_error(resultado_modelos['real_y'], resultado_modelos['prediccion_ann'])
mse_svr=mean_squared_error(resultado_modelos['real_y'], resultado_modelos['prediccion_svr'])

rmse_ann=math.sqrt(mse_ann)
rmse_svr=math.sqrt(mse_svr)

from sklearn.metrics import mean_absolute_error
mae_ann=mean_absolute_error(resultado_modelos['real_y'], resultado_modelos['prediccion_ann'])
mae_svr=mean_absolute_error(resultado_modelos['real_y'], resultado_modelos['prediccion_svr'])


### Matriz de Mediciones de Error

In [ ]:
import seaborn as sns
cm = sns.light_palette("palegreen", as_cmap=True, reverse=True)

matriz_comparativa = [['SVR',mse_svr,rmse_svr,mae_svr],['ANN',mse_ann,rmse_ann,mae_ann]]
matriz_comparativa = pd.DataFrame(matriz_comparativa, columns =['Algoritmo', 'MSE', 'RMSE', 'MAE']) 
matriz_comparativa.set_index('Algoritmo')

print ("Medidas de error - COLCAP")
print("")
matriz_comparativa.style.background_gradient(cmap=cm)

### Matriz de Desviaciones Porcentuales

In [ ]:
min_MSE = matriz_comparativa['MSE'].min()
min_RMSE = matriz_comparativa['RMSE'].min()
min_MAE = matriz_comparativa['MAE'].min()

matriz_comparativa['Desv MSE'] = (matriz_comparativa['MSE']-min_MSE)/min_MSE
matriz_comparativa['Desv RMSE'] = (matriz_comparativa['RMSE']-min_RMSE)/min_RMSE
matriz_comparativa['Desv MAE'] = (matriz_comparativa['MAE']-min_MAE)/min_MAE

matriz_desviaciones=matriz_comparativa.drop(columns=['MSE', 'RMSE', 'MAE'])

matriz_desviaciones.set_index('Algoritmo')

print ("Medidas de desviación - COLCAP")
print("")
matriz_desviaciones.style.background_gradient(cmap=cm)

### Desviación Media por Algoritmo

In [ ]:
matriz_desviaciones['desv_media'] = matriz_desviaciones.mean(axis = 1, skipna = True)
desv_media=matriz_desviaciones.drop(columns=['Desv MSE', 'Desv RMSE', 'Desv MAE'])
desv_media.set_index('Algoritmo')

print ("Desviación media - COLCAP")
print("")
desv_media.style.background_gradient(cmap=cm)

# Selección del Modelo

In [ ]:
desv_media['mejor_algoritmo'] = desv_media['desv_media']==desv_media['desv_media'].min()
mejor_algoritmo = desv_media[desv_media['mejor_algoritmo']==True]
mejor_algoritmo=mejor_algoritmo.drop(columns=['desv_media', 'mejor_algoritmo'])
mejor_algoritmo.head()